In [ ]:
import torch
import os
import pandas as pd
import albumentations as A
import numpy as np
import matplotlib.pyplot as plt
import cv2
import random

c=0
l=[]
s=["hue","gaussian","gauss"]
for i in os.listdir("D:/cat_id/data/raw/train_data/yolo/images"):
    l.append(i)                 # To take images randomly, we are appending the names of all images to a list and then shuffling it 
random.shuffle(l) 
for i in l:
    j=i.split(".")[0]
    d=0
    for k in s:
        if k in j:
            d=1        #We are checking if the image is previously augmented or not. If it is previously augmented, we are ignoring it
    if d==0:
        # Define the transformation pipeline
        transform = A.Compose([
            # A.Resize(512, 512),
            # A.HorizontalFlip(p=1),
            # A.RandomCrop(width=400, height=300),
            # A.RandomBrightnessContrast(p=0.5),
            #   A.Blur(p=1),
            # A.HueSaturationValue(p=1),
            # A.GaussianBlur(p=1),
            # A.RandomSizedBBoxSafeCrop(height=200,width=100),
            # A.Transpose(p=1)
            # A.Crop(x_min=0,y_min=200,x_max=500,y_max=500,p=1),
            A.Rotate(p=1,limit=[340,340]),
            # A.VerticalFlip(p=0.5),
            # A.RandomRotate90(p=1),
            # A.RandomGridShuffle(p=0.5),
            # A.Normalize(),
            # zoom for front facing images
            # horizontal flip for side facing images
        ], bbox_params=A.BboxParams(format="pascal_voc",label_fields=['labels']))    # Based on the labels that we have we need to decide the format.

        # Load and preprocess an example image and bounding box coordinates
        # la=pd.read_csv("D:/cat_id/data_structure/raw/train_data/yolo/labels/00000026_008.txt")
        try:
            la=pd.read_csv(f"D:/cat_id/data/raw/train_data/yolo/labels/{j}.txt")  # reading the labels txt file using pandas 
            z=int(la.columns.values[0].split(" ")[0])
            if z==0:
                la=la.columns.values[0].split(" ")[1:]
                la=[float(k) for k in la]
                image = cv2.imread(f"D:/cat_id/data/raw/train_data/yolo/images/{j}.jpg")  # 
                # results=model(image)
                # eye_numbered=0
                # for i in results.xyxy[0]:                # if we want to see number of eyes for the cat present in the image
                #     cl=i[-1]
                #     if cl==0:                                                
                #         eye_numbered=eye_numbered+1
                # if eye_numbered==2:
                    
                bboxes = [la] 
                if z==0:
                    labels = ["cat"]
                if z==1:
                    labels=["dog"]  # Example class labels


                xmin=max(0,int(((la[0]*image.shape[1])-(la[2]*image.shape[1])/2)))
                ymin=max(0,int(((la[1]*image.shape[0])-(la[3]*image.shape[0])/2)))
                xmax=max(0,int(((la[0]*image.shape[1])+(la[2]*image.shape[1])/2)))
                ymax=max(0,int(((la[1]*image.shape[0])+(la[3]*image.shape[0])/2)))

                image1=image[ymin:ymin+(ymax-ymin)+40,xmin:xmin+(xmax-xmin)+40]
                n_xmin=0
                n_ymin=0
                n_xmax=image1.shape[1]-40
                n_ymax=image1.shape[0]-40
                n_bboxes=[[n_xmin,n_ymin,n_xmax,n_ymax]]

                # Apply the transformation pipeline to the image and bounding box coordinates
                transformed = transform(image=image1, bboxes=n_bboxes, labels=labels)   #apply augmentations to our image

                # Retrieve the transformed image and updated bounding box coordinates
                transformed_image = transformed['image']
                transformed_bboxes = transformed['bboxes']
                transformed_labels = transformed['labels']

                xmin1=int(transformed_bboxes[0][0])
                ymin1=int(transformed_bboxes[0][1])
                xmax1=int(transformed_bboxes[0][2])
                ymax1=int(transformed_bboxes[0][3])

                center_x=(xmin1 + xmax1) / 2        
                center_y=(ymin1+ymax1)/2
                width=xmax1-xmin1
                height=ymax1-ymin1
                center_x_normalized = center_x / transformed_image.shape[1]     # converting to pascal coordinates into yolo coordinates
                center_y_normalized = center_y / transformed_image.shape[0]
                width_normalized = width / transformed_image.shape[1]
                height_normalized=height/transformed_image.shape[0]
                cv2.imwrite(f"albumentations/cats/images_cats_rotate_340_same_aspect/{j}_rotate_340_same_aspect.jpg",transformed_image)

                with open(f"albumentations/cats/labels_cats_rotate_340_same_aspect/{j}_rotate_340_same_aspect.txt","w") as file:    # To write the new augmented image labels into a file.
                    if transformed_labels==["cat"]:         
                        file.write(str(0))
                    if transformed_labels==["dog"]:
                        file.write(str(1))
                    file.write(" ")
                    file.write(str(np.round(center_x_normalized,decimals=2)))
                    file.write(" ")
                    file.write(str(np.round(center_y_normalized,decimals=2)))
                    file.write(" ")
                    file.write(str(np.round(width_normalized,decimals=2)))
                    file.write(" ")
                    file.write(str(np.round(height_normalized,decimals=2)))

                c=c+1
                if c==2000:
                    break       #to set the limit of the augmentations of number of images. 
        except:
            print(0,j)